# Installation and Imports

In [ ]:
pip install -U sentence-transformers


     |████████████████████████████████| 78 kB 2.9 MB/s 
     |████████████████████████████████| 3.5 MB 25.5 MB/s 
     |████████████████████████████████| 6.8 MB 26.5 MB/s 
     |████████████████████████████████| 1.2 MB 38.6 MB/s 
     |████████████████████████████████| 67 kB 2.6 MB/s 
     |████████████████████████████████| 895 kB 40.8 MB/s 
     |████████████████████████████████| 596 kB 33.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=92dfd94cea3f5d78c984cc6ea3dd169f2f8dbd3c57007096a7ff9ff0b89ac7a7
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from torch.utils.data import DataLoader
import math
import logging
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk, re, string, collections
from nltk.util import ngrams
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import gensim
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import sklearn
from sklearn.model_selection import train_test_split
import os
import gzip
import csv
import random
import keras

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Dataset Load

Loading the CSV

In [ ]:
fake = pd.read_csv('./Fake.csv')
real = pd.read_csv('./Real.csv')

In [ ]:
fake['Target']=0
real['Target']=1

In [ ]:
frames = [fake, real]

df = pd.concat(frames)

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df_=df.copy()
df_=df_.sort_values(by=['date'])
df_=df_.reset_index(drop=True)


In [ ]:
df_['news']=df_['subject']+' '+df_['title']+' '+df_['text']
df_['news'] = df_.apply(lambda x: x['news'].lower(),axis=1)
df_["news"] = df_['news'].str.replace('[^\w\s]','')
all_news=pd.DataFrame(pd.Series(' '.join(df_['news']).split()).value_counts())
allnews1=all_news.head(30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df_['news'], 
                                                    df_['Target'], 
                                                    random_state=0)

In [ ]:
X_train

13637    politics brilliant newt gingrich heres why tru...
42421    worldnews its not over yet merkel warns suppor...
37916    worldnews turkish court releases eight rights ...
3321     leftnews epic news conference about milwaukee ...
2345     news  white house staffer admits this ss hard ...
                               ...                        
30403    worldnews militants cross from afghanistan kil...
21243    news  greedy trump cuts takes housing money aw...
42613    worldnews banks wont be allowed to do business...
43567    politicsnews white house calls senate override...
2732     middleeast members ep 5  drive by wire taxi to...
Name: news, Length: 33673, dtype: object

In [ ]:
np.array(X_train)

array(['politics brilliant newt gingrich heres why trump should not trust intel community right now video wouldn t you know that the cia director had words for trump today brennan has some nerve  cautioning  presidentelect trumpwashington reuters   cia director john brennan on sunday had a stern parting message for republican donald trump days before he assumes the us presidency cautioning him against loosening sanctions on russia and warning him to watch what he says',
       'worldnews its not over yet merkel warns supporters before vote hamburg reuters  returning to her birthplace chancellor angela merkel urged supporters to mobilize undecided voters before sunday s election telling them not to take her conservatives  solid lead in opinion polls for granted surveys show many voters are still undecided and with the election likely to return a fractured parliament she wants to maximize support to bolster her position in coalition negotiations  what i ve heard in the last days is unbel

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")


model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings_train = model.encode(np.array(X_train))

embeddings_test = model.encode(np.array(X_test))

In [ ]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

feature_vector_length = 384
num_classes = 2

input_shape = (feature_vector_length,)

Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(350, input_shape=input_shape, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(embeddings_train, Y_train, epochs=10, batch_size=250, verbose=1, validation_split=0.2)

Epoch 1/10
108/108 [==============================] - 3s 8ms/step - loss: 0.2374 - accuracy: 0.9108 - val_loss: 0.1287 - val_accuracy: 0.9512
Epoch 2/10
108/108 [==============================] - 1s 5ms/step - loss: 0.1157 - accuracy: 0.9549 - val_loss: 0.1029 - val_accuracy: 0.9605
Epoch 3/10
108/108 [==============================] - 1s 5ms/step - loss: 0.0917 - accuracy: 0.9660 - val_loss: 0.0927 - val_accuracy: 0.9627
Epoch 4/10
108/108 [==============================] - 1s 6ms/step - loss: 0.0738 - accuracy: 0.9739 - val_loss: 0.0919 - val_accuracy: 0.9647
Epoch 5/10
108/108 [==============================] - 1s 5ms/step - loss: 0.0619 - accuracy: 0.9778 - val_loss: 0.0795 - val_accuracy: 0.9722
Epoch 6/10
108/108 [==============================] - 1s 5ms/step - loss: 0.0436 - accuracy: 0.9856 - val_loss: 0.0752 - val_accuracy: 0.9743
Epoch 7/10
108/108 [==============================] - 1s 5ms/step - loss: 0.0358 - accuracy: 0.9881 - val_loss: 0.0784 - val_accuracy: 0.9746
Epoch 

In [ ]:
test_results = model.evaluate(embeddings_test, Y_test, verbose=1)

351/351 [==============================] - 2s 5ms/step - loss: 0.0904 - accuracy: 0.9731
